In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
path = "/content/drive/MyDrive/연구실/"

In [3]:
!pip install git+https://github.com/hyperopt/hyperopt.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/hyperopt/hyperopt.git to /tmp/pip-req-build-_v8o0ecw
  Running command git clone --filter=blob:none --quiet https://github.com/hyperopt/hyperopt.git /tmp/pip-req-build-_v8o0ecw
  Resolved https://github.com/hyperopt/hyperopt.git to commit 6c5b33be93a71a4ae6a38f04a29b18020b78b702
  Running command git submodule update --init --recursive -q
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 KB 5.3 MB/s eta 0:00:00
  Created wheel for hyperopt: filename=hyperopt-0.2.7-py2.py3-none-any.whl size=966138 sha256=a69f7bc8b84a687641790db1f2ee8e84a1fdb2abf9b04f8615d56c88223151c1
  Stored in directory: /tmp/pip-ephem-wheel-cache-jowkfmb_/wheels/7e/a2/bc/da25ffbedcf76e6957370dd98f8bfc0b9cda94cef6c350d117
Successfully built hyperopt
  Attempting uninstall: hyperopt
    Found existing installation: hyperopt 0.1.2
    Uninstallin

In [4]:
import pandas as pd
import numpy as np

from hyperopt import hp
from hyperopt import STATUS_OK
from hyperopt.early_stop import no_progress_loss

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from tqdm import tqdm
import joblib

import warnings
warnings.filterwarnings("ignore")

In [5]:
data = pd.read_csv(path+'전처리최종이었음좋겠다.csv')
data.columns

Index(['yearmonth', 'Ticker', '한국표준산업분류코드10차(중분류)', '수익률 (1개월)(%)',
       '베타 (D,1Yr)', 'PER(배)', 'PSR(배)', 'chmom', 'mom36m', 'zerotrade',
       'std_turn', 'mean_ill', 'retvol', 'maxret', 'var_market', 'mom1m',
       'mom12m', 'indmom', 'turn', 'mvel1', 'dolvol', 'betasq', 'Idiovol',
       'macro_(콜금리-기준금리)평균', 'macro_원/달러환율', 'macro_금($/OZ)', 'macro_뉴스심리지수',
       'macro_KOSPI dp', 'macro_KOSPI PER', 'macro_KOSPI PBR', 'macro_CD Rate',
       'macro_Term spread', 'macro_Default Spread', 'macro_VKOSPI 200'],
      dtype='object')

In [6]:
X = data.drop(['한국표준산업분류코드10차(중분류)', '수익률 (1개월)(%)', 'macro_뉴스심리지수'], axis=1)
y = data['수익률 (1개월)(%)']
ym = data['yearmonth']
tk = data['Ticker']

In [ ]:
X.shape

(312185, 31)

In [9]:
a = '2004|2005|2006|2007|2008|2009|2010|2011|2012|2013|2014|2015|2016|2017|2018|2019|2020'
ya0 = []
tk0 = []
ym0 = []
ya1 = []
sse = 0

for i in tqdm(range(9)): 
    # tX = X[ym.str.contains(a[:24+5*i])]
    # vX = X[ym.str.contains(a[25+5*i:39+5*i])]
    tvX = X[ym.str.contains(a[:39+5*i])]
    tvym = ym[ym.str.contains(a[:39+5*i])]
    oX = X[ym.str.contains(a[40+5*i:44+5*i])]
    # ty = y[ym.str.contains(a[:24+5*i])]
    Vy = y[ym.str.contains(a[25+5*i:39+5*i])]
    tvy = y[ym.str.contains(a[:39+5*i])]
    oy = y[ym.str.contains(a[40+5*i:44+5*i])]
    otk = tk[ym.str.contains(a[40+5*i:44+5*i])]
    oym = ym[ym.str.contains(a[40+5*i:44+5*i])]
    
    # 하이퍼파라미터 검색 공간 설정
    pcr_search_space = {
        'n_components' : hp.quniform('n_components', 1, 29, 1)
    }
    
    # 목적 함수 정의
    def objective_func(search_space):
        pcr = Pipeline([
            ('scaler', StandardScaler()),
            ('PCR', PCA(n_components = int(search_space['n_components']))),
            ('LR', LinearRegression())
        ])
        
        d = 0
        
        # time series cv
        for j in range(3): # 마지막 3년만 valid로 사용
            tX = X[ym.str.contains(a[:24+5*(j+i)])]
            vX = X[ym.str.contains(a[25+5*(j+i):29+5*(j+i)])]
            ty = y[ym.str.contains(a[:24+5*(j+i)])]
            vy = y[ym.str.contains(a[25+5*(j+i):29+5*(j+i)])]
            
            # for문에 필요한 key를 미리 만듦
            ym_temp = vX['yearmonth'].reset_index(drop=True) # t = ym_temp==k를 위해 reset index
            key_yearmonth = dict.fromkeys(ym_temp)
            
            # vy의 index를 reset
            vy.reset_index(drop=True, inplace=True) # r = r*(vy[sort[-d:].index].mean()+100)/100를 위해 reset index
            
            # 년도, 티커 드랍
            tX = tX.drop(['yearmonth', 'Ticker'], axis=1)
            vX = vX.drop(['yearmonth', 'Ticker'], axis=1)
            
            # fitting
            pcr.fit(tX, ty)
            
            # r2oos 계산         
            y_pred = pd.Series(pcr.predict(vX).reshape(-1,)) # sort_values() 사용을 위해 변환
            d = d + sum((vy-y_pred)**2) 
                
        e = sum(Vy**2)
        r2oos = 1-(d/e)
        new_loss = r2oos*(-1)
        
        return {
            'loss':new_loss,
            'status':STATUS_OK
        }
    
    # fmin()을 사용하여 최적 하이퍼 파라미터 찾기
    from hyperopt import fmin, tpe, Trials
    import warnings
    warnings.filterwarnings('ignore')

    trial_val = Trials() # 결과 저장
    
    best = fmin(
        fn=objective_func,
        space=pcr_search_space,
        algo=tpe.suggest,
        max_evals=1000,    # 최대 반복 횟수 지정
        trials=trial_val,
        return_argmin=False, # hp.choice의 경우 값 대신 index를 반환함 
        early_stop_fn=no_progress_loss(20), # 30회 동안 개선되지 않으면 조기중지
        # https://github.com/hyperopt/hyperopt/issues/689
        rstate=np.random.default_rng(42) # https://github.com/hyperopt/hyperopt/issues/838
    )
    
    # 정수형 하이퍼 파라미터 값이 뒤에 .0이 붙어서 실수형으로 반환됨을 유의하기!
    best['n_components'] = int(best['n_components'])
    print(best) 
    
    reg0 = Pipeline([
            ('scaler', StandardScaler()),
            ('PCR', PCA(**best)),
            ('LR', LinearRegression())
        ])
    
    import warnings
    warnings.filterwarnings('ignore')
    
    tvX = tvX.drop(['yearmonth', 'Ticker'], axis=1)
    oX = oX.drop(['yearmonth', 'Ticker'], axis=1)
    
    reg0.fit(tvX, tvy)
    
    yy0 = reg0.predict(oX)
    ya0 = ya0 + oy.tolist()
    ya1 = ya1 + yy0.tolist()
    tk0 = tk0 + otk.tolist()
    ym0 = ym0 + oym.tolist()
    yy = yy0.reshape(-1,)
    joblib.dump(reg0,path+'PCR_r2oos_'+str(i)+'.pkl')
    print(1-sum((oy-yy)**2)/sum(oy**2))
    sse = sse + sum((oy-yy)**2)

oyy = y[ym.str.contains(a[40:84])]
e = sum(oyy**2)
r2oos = 1-(sse/e)
print(r2oos)

PCR = pd.DataFrame({'real':ya0,'predict':ya1,'ticker':tk0,'yearmonth':ym0})
PCR.to_csv(path+'PCR_r2oos.csv')  

  0%|          | 0/9 [00:00<?, ?it/s]


  3%|▎         | 34/1000 [02:25<1:09:04,  4.29s/trial, best loss: -0.02425079152635745]
{'n_components': 3}


 11%|█         | 1/9 [02:27<19:42, 147.84s/it]

0.009740245589336727

  2%|▏         | 23/1000 [01:47<1:16:23,  4.69s/trial, best loss: -0.012038517832473117]
{'n_components': 5}


 22%|██▏       | 2/9 [04:17<14:39, 125.58s/it]

...

 89%|████████▉ | 8/9 [29:59<04:32, 272.17s/it]

-0.024972804432118245

  6%|▌         | 61/1000 [07:50<2:00:38,  7.71s/trial, best loss: 0.004042023332336164]
{'n_components': 3}


100%|██████████| 9/9 [37:53<00:00, 252.59s/it]

0.014824414904506256


-0.0002956328309737799


In [11]:
ya0 = pd.Series(PCR['real'])
ya1 = pd.Series(PCR['predict'])
tk0 = pd.Series(PCR['ticker'])
ym0 = pd.Series(PCR['yearmonth'])

r1 = []
r2 = []
for i in list(dict.fromkeys(ym0)):
    t = ym0==i
    d = round(len(ya1[t])/10)
    sort = ya1[t].sort_values()
    r1.append(ya0[sort[-d:].index].mean())
    r2.append(ya0[sort[0:d].index].mean())

rr1 = 1
rpcr1 = [1]
for i in r1:
    rr1 = rr1*(i+100)/100
    rpcr1.append(rr1)

In [12]:
rpcr1[-1]

7.312361661058831